In [1]:
# !/usr/bin/python
# -*- coding:utf-8 -*-

import numpy as np
from gensim import corpora, models, similarities
from pprint import pprint
import time


# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
def load_stopword():
    f_stop = open('../dataset/stopword.txt')
    sw = [line.strip() for line in f_stop]
    f_stop.close()
    return sw


print '初始化停止词列表 --'
t_start = time.time()
stop_words = load_stopword()

print '开始读入语料数据 -- '
f = open('../dataset/news.dat')  # LDA_test.txt
texts = [[word for word in line.strip().lower().split() if word not in stop_words] for line in f]
# texts = [line.strip().split() for line in f]
print '读入语料数据完成，用时%.3f秒' % (time.time() - t_start)
f.close()
M = len(texts)
print '文本数目：%d个' % M
# pprint(texts)

print '正在建立词典 --'
dictionary = corpora.Dictionary(texts)
V = len(dictionary)
print u'词的个数：', V
print '正在计算文本向量 --'
corpus = [dictionary.doc2bow(text) for text in texts]
print '正在计算文档TF-IDF --'
t_start = time.time()
corpus_tfidf = models.TfidfModel(corpus)[corpus]
print '建立文档TF-IDF完成，用时%.3f秒' % (time.time() - t_start)

初始化停止词列表 --
开始读入语料数据 -- 
读入语料数据完成，用时10.877秒
文本数目：2043个
正在建立词典 --
词的个数： 64326
正在计算文本向量 --
正在计算文档TF-IDF --
建立文档TF-IDF完成，用时0.147秒


In [4]:
print 'LDA模型拟合推断 --'
num_topics = 10
t_start = time.time()
lda = models.LdaModel(corpus_tfidf, num_topics=num_topics, id2word=dictionary,
                      alpha=0.01, eta=0.01, minimum_probability=0.001,
                      update_every=1, chunksize=100, passes=1)
print 'LDA模型完成，训练时间为\t%.3f秒' % (time.time() - t_start)

LDA模型拟合推断 --
LDA模型完成，训练时间为	18.090秒


In [5]:
# 所有文档的主题
doc_topic = [a for a in lda[corpus_tfidf]]
print 'Document-Topic:\n'
pprint(doc_topic)

Document-Topic:

[[(0, 0.0043802826962712005),
  (1, 0.0011235474786826186),
  (2, 0.0011235474786826186),
  (3, 0.16499873845284313),
  (4, 0.0055138654921372595),
  (5, 0.019785358793107867),
  (6, 0.65604401533829848),
  (7, 0.091791449972784087),
  (8, 0.0011235474786826186),
  (9, 0.054115646818510194)],
 [(0, 0.47015466083872282),
  (1, 0.0098894676155815327),
  (2, 0.012305167699077494),
  (3, 0.20742201220453188),
  (4, 0.02750609691750518),
  (5, 0.093237580538000839),
  (6, 0.013260356299704538),
  (7, 0.091219710030004161),
  (8, 0.044717298007264865),
  (9, 0.030287649849606659)],
 [(1, 0.0030701329127748754),
  (2, 0.0040274228864969307),
  (3, 0.16101055120730681),
  (5, 0.022863515637077461),
  (6, 0.0031772835482643961),
  (7, 0.8010337635400413),
  (9, 0.0023881133893796629)],
 [(0, 0.0019913191887715137),
  (1, 0.61543365674565764),
  (2, 0.0019913191887715137),
  (3, 0.18836113852324851),
  (4, 0.0019913191887715137),
  (5, 0.0019913191887715137),
  (6, 0.00199131918

In [7]:
# 随机打印某10个文档的主题
num_show_topic = 10  # 每个文档显示前几个主题
print '10个文档的主题分布：'
doc_topics = lda.get_document_topics(corpus_tfidf)  # 所有文档的主题分布
idx = np.arange(M)
np.random.shuffle(idx)
idx = idx[:10]
for i in idx:
    topic = np.array(doc_topics[i])
    print 'topic = \t', topic
    topic_distribute = np.array(topic[:, 1])
    # print topic_distribute
    topic_idx = topic_distribute.argsort()[:-num_show_topic - 1:-1]
    print ('第%d个文档的前%d个主题：' % (i, num_show_topic)), topic_idx
    print topic_distribute[topic_idx]

10个文档的主题分布：
topic = 	[[ 0.          0.19940754]
 [ 1.          0.03595037]
 [ 2.          0.03738872]
 [ 3.          0.24698319]
 [ 4.          0.05597035]
 [ 5.          0.30228991]
 [ 6.          0.02974045]
 [ 7.          0.02458058]
 [ 8.          0.02233648]
 [ 9.          0.04535241]]
第1349个文档的前10个主题： [5 3 0 4 9 2 1 6 7 8]
[ 0.30228991  0.24698319  0.19940754  0.05597035  0.04535241  0.03738872
  0.03595037  0.02974045  0.02458058  0.02233648]
topic = 	[[  0.00000000e+00   1.35108458e-03]
 [  1.00000000e+00   1.35108458e-03]
 [  2.00000000e+00   1.35108458e-03]
 [  3.00000000e+00   9.35436982e-01]
 [  4.00000000e+00   9.30055672e-03]
 [  5.00000000e+00   1.35108458e-03]
 [  6.00000000e+00   1.35108458e-03]
 [  7.00000000e+00   4.58048695e-02]
 [  8.00000000e+00   1.35108458e-03]
 [  9.00000000e+00   1.35108458e-03]]
第87个文档的前10个主题： [3 7 4 9 8 6 5 2 1 0]
[ 0.93543698  0.04580487  0.00930056  0.00135108  0.00135108  0.00135108
  0.00135108  0.00135108  0.00135108  0.00135108]
topic 

In [8]:
num_show_term = 7  # 每个主题显示几个词
print '每个主题的词分布：'
for topic_id in range(num_topics):
    print '主题#%d：\t' % topic_id
    term_distribute_all = lda.get_topic_terms(topicid=topic_id)
    term_distribute = term_distribute_all[:num_show_term]
    term_distribute = np.array(term_distribute)
    term_id = term_distribute[:, 0].astype(np.int)
    print '词：\t',
    for t in term_id:
        print dictionary.id2token[t],
    print
    # print '\n概率：\t', term_distribute[:, 1]

每个主题的词分布：
主题#0：	
词：	日本 俄罗斯 林 土耳其 普京 布尔 北部
主题#1：	
词：	督察 环保 督察组 县级市 国务院 大同市 意大利
主题#2：	
词：	星巴克 越南 南京 寺庙 陈某 渔船 拆迁
主题#3：	
词：	中国 医院 和 了 发展 是 他
主题#4：	
词：	学生 警方 雷洋 学校 死亡 案件 儿子
主题#5：	
词：	朝鲜 美国 特朗普 总统 菲律宾 经济 金正恩
主题#6：	
词：	拆违 暴力 村民 列车 睾丸 嫖娼 男生
主题#7：	
词：	阅读 公司 游客 读书 南海 投资 仲裁
主题#8：	
词：	胡 殴打 右肾 嫌犯 陈满 手术 转业
主题#9：	
词：	她 孩子 我 父母 城市 结婚 王某
